In [ ]:
from htm_rl.agent.agent import Agent, AgentRunner
from htm_rl.agent.memory import Memory, TemporalMemory
from htm_rl.agent.planner import Planner
from htm_rl.common.sa_sdr_encoder import SaSdrEncoder, format_sa_superposition
from htm_rl.common.base_sa import SaRelatedComposition, Sa, SaSuperposition
from htm_rl.common.int_sdr_encoder import IntSdrEncoder, IntRangeEncoder
from htm_rl.common.int_sdr_encoder import SequenceSdrEncoder
from htm_rl.envs.gridworld_pomdp import GridWorld

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint

In [ ]:
def init_memory(pars, sa_encoder, start_indicator=None, output_file=None):
    tm = TemporalMemory(**pars)
    memory = Memory(tm, sa_encoder, sa_encoder.format, format_sa_superposition,
                    start_indicator=start_indicator, output_file=output_file)
    return memory

In [ ]:
actions = [2, 2, 1, 2, 2, 1, 2, 2]
world_description = [[2,0,0],
                     [1,1,0],
                     [0,0,0]]

In [ ]:
gw = GridWorld(world_description, (3, 3), agent_initial_position={'row': 2, 'column': 0},
               observable_vars=['flatten_index', 'direction'])

In [ ]:
gw.render()

In [ ]:
gw.observable_state, gw.filtered_observation

In [ ]:
index_bits = 2
direction_bits = 2
action_bits = 2

state_encoder = SequenceSdrEncoder('state',
                                   encoders=[
                                             IntSdrEncoder('flatten_index',
                                                                   gw.world_size[0]*gw.world_size[1]+1,
                                                                   index_bits,
                                                                   index_bits),
                                             IntSdrEncoder('direction', 4+1, direction_bits, direction_bits-1)
                                            ],
                                   size=2)

In [ ]:
action_encoder = IntSdrEncoder('action', gw.n_actions+1,
                              value_bits=action_bits, activation_threshold=action_bits)

In [ ]:
sa_encoder = SaSdrEncoder(state_encoder, action_encoder)

In [ ]:
sa_encoder.total_bits, sa_encoder.value_bits, sa_encoder.activation_threshold

In [ ]:
goal_state = (1, 2)
goal_state

In [ ]:
pars = dict(n_columns=sa_encoder.total_bits,
                                cells_per_column=1,
                                activation_threshold=sa_encoder.value_bits,
                                learning_threshold=sa_encoder.value_bits,
                                initial_permanence=0.51,
                                connected_permanence=0.5,
                                maxNewSynapseCount=sa_encoder.value_bits,
                                maxSynapsesPerSegment=sa_encoder.value_bits,
                                permanenceIncrement=0.1,
                                permanenceDecrement=0.05,
                                predictedSegmentDecrement=0.001)

In [ ]:
memory = init_memory(pars, sa_encoder, start_indicator=Sa((9, 4), 3))

In [ ]:
planner = Planner(memory, 5, 1)

In [ ]:
agent = Agent(memory, planner, gw.n_actions)

In [ ]:
max_steps = 25
run = AgentRunner(agent, gw, 500, max_steps, 50, 1)

In [ ]:
run.run()

In [ ]:
run.agent.planner.inter_episode_goal_memory._set

In [ ]:
run.verbosity = 3
run.n_episodes = 1
run.pretrain = 0
run.run()

In [ ]:
fig = plt.figure(figsize=(10, 7))
steps = np.array(run.train_stats.steps[:])
plt.plot(np.arange(steps.size), steps, '.')